Importing dependencies

In [589]:
import pandas as pd
import datetime
import numpy as np

Importing Data

In [590]:
raw_data = pd.read_csv(r'BC_LCG_2017_CN.csv')
additional_info_1 = pd.read_excel(r'BC_LCG_2017_CN.xlsb', sheet_name='IA1', index_col=0)
additional_info_2 = pd.read_excel(r'BC_LCG_2017_CN.xlsb', sheet_name='IA2',index_col=0)

In [591]:
#checking data
cleaned = raw_data
cleaned.dropna(how='all', axis=0, inplace=True)
cleaned.head()

,Fecha,Número de Vendedor,Unnamed: 2,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo,Unnamed: 12,Unnamed: 13
0,1/01/2015,7.0,NaN,NaN,66.0,Real,21.0,NaN,328.0,TRATAMIENTO DE CONCRETO,31740.12,22046.54,NaN,NaN
1,1/01/2015,5.0,NaN,NaN,45.0,Real,21.0,NaN,370.0,PINTURAS DECORATIVAS,2377.30,1761.35,NaN,NaN
2,1/01/2015,8.0,NaN,NaN,75.0,Real,NaN,Pinturas,371.0,BARNICES ARQUITECTÓNICOS,20422.48,13189.05,NaN,NaN
3,1/01/2015,6.0,NaN,NaN,57.0,Real,21.0,NaN,372.0,PINTURA AUTOMOTRIZ,719.80,668.40,NaN,NaN
4,1/01/2015,7.0,NaN,NaN,62.0,Real,21.0,NaN,376.0,TINTES PARA MADERA,15589.78,10835.03,NaN,NaN


In [592]:
#creating dict for additional info
add_inf_dict1 = {}
for a,b in additional_info_1.iterrows():
    add_inf_dict1[b.name] = b.Departamento

add_inf_dict1_inverse = {v: k for k, v in add_inf_dict1.items()}
print(add_inf_dict1)
print(add_inf_dict1_inverse)

add_inf_dict2 = {}
for a,b in additional_info_2.iterrows():
    add_inf_dict2[b.name] = b.Nombre + ' ' + b.Apellido

{15: 'Alfombras', 806: 'Artículos Impulso', 7: 'Artículos Navideños', 25: 'Automotriz', 8: 'Baños', 11: 'Bebes', 2: 'Blancos', 5: 'Cocina', 42: 'Cortinaje', 2250: 'Decoracion', 22: 'Eléctricos', 3: 'Electrodomésticos', 58: 'Electronica', 26: 'Ferretería', 2251: 'Guatemala nuestra', 23: 'Herramientas', 27: 'Jardinería', 1: 'Juguetes', 10: 'Lámparas', 12: 'Librería', 30: 'Limpieza', 2248: 'Mascotas', 50: 'Materiales de Construcción', 4: 'Mesa', 59: 'Muebles de Exterior', 2249: 'Muebles de Interior', 807: 'Niños', 9: 'Organización', 28: 'Out Door Living', 21: 'Pinturas', 24: 'Plomería', 2247: 'Seguridad'}
{'Alfombras': 15, 'Artículos Impulso': 806, 'Artículos Navideños': 7, 'Automotriz': 25, 'Baños': 8, 'Bebes': 11, 'Blancos': 2, 'Cocina': 5, 'Cortinaje': 42, 'Decoracion': 2250, 'Eléctricos': 22, 'Electrodomésticos': 3, 'Electronica': 58, 'Ferretería': 26, 'Guatemala nuestra': 2251, 'Herramientas': 23, 'Jardinería': 27, 'Juguetes': 1, 'Lámparas': 10, 'Librería': 12, 'Limpieza': 30, 'Masco

Adding missing data and droppig empty rows/columns

In [593]:
#seller names
seller_names = []
for code in cleaned['Número de Vendedor']:
    seller_names.append(add_inf_dict2[code])
cleaned['Nom_Completo_Vendedor'] = seller_names
cleaned.dropna(how='all', axis=1, inplace=True)
cleaned = cleaned.where(pd.notnull(cleaned), '0')
cleaned['Departamento - Clave'] = cleaned['Departamento - Clave'].astype('int')
cleaned.head()

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo
0,1/01/2015,7,Carlos Vela,66,Real,21,0,328,TRATAMIENTO DE CONCRETO,31740.12,22046.54
1,1/01/2015,5,Gonzalo Aguilar,45,Real,21,0,370,PINTURAS DECORATIVAS,2377.3,1761.35
2,1/01/2015,8,Ernesto Pineda,75,Real,0,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,13189.05
3,1/01/2015,6,Paulina Casanova,57,Real,21,0,372,PINTURA AUTOMOTRIZ,719.8,668.4
4,1/01/2015,7,Carlos Vela,62,Real,21,0,376,TINTES PARA MADERA,15589.78,10835.03


In [594]:
#departmento
dept_codes = list(cleaned['Departamento - Clave'])
dept_names = list(cleaned['Departamento'])
dept_names_new = []
#print('Len codes: ',len(dept_codes))
#print('Len names: ',len(dept_names))

for i, row in enumerate(dept_codes):
    if row == 0:
        try: 
            dept_codes[i] = add_inf_dict1_inverse[dept_names[i]]
        except:
            print('Exception at Code index: ', i, row)
            dept_codes[i] = 'Inf. faltante'

for row in dept_codes:
    if row == 'Inf. faltante':
        dept_names_new.append('Inf. faltante')
    else:
        dept_names_new.append(add_inf_dict1[row])

#print('Len codes: ',len(dept_codes))
#print('Len names: ',len(dept_names_new))

cleaned['Departamento - Clave'] = dept_codes
cleaned['Departamento'] = dept_names_new
cleaned.head()

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo
0,1/01/2015,7,Carlos Vela,66,Real,21,Pinturas,328,TRATAMIENTO DE CONCRETO,31740.12,22046.54
1,1/01/2015,5,Gonzalo Aguilar,45,Real,21,Pinturas,370,PINTURAS DECORATIVAS,2377.3,1761.35
2,1/01/2015,8,Ernesto Pineda,75,Real,21,Pinturas,371,BARNICES ARQUITECTÓNICOS,20422.48,13189.05
3,1/01/2015,6,Paulina Casanova,57,Real,21,Pinturas,372,PINTURA AUTOMOTRIZ,719.8,668.4
4,1/01/2015,7,Carlos Vela,62,Real,21,Pinturas,376,TINTES PARA MADERA,15589.78,10835.03


In [595]:
#Calculating PARETO ABCs

['Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Pinturas', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Eléctricos', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Herramientas', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Plomería', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Automotriz', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Ferretería', 'Fe